**BUAN6341: Assignment 3 **

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyodbc 
import multiprocessing
import os
%matplotlib inline
import seaborn as sns
from sklearn import svm
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import learning_curve, cross_val_score
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error,confusion_matrix, roc_curve,roc_auc_score, accuracy_score, homogeneity_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import FastICA, PCA
from sklearn.random_projection import GaussianRandomProjection
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import learning_curve, cross_val_score, train_test_split,GridSearchCV,learning_curve, KFold

In [ ]:
conda deactivate  

pip install tensorflow 

In [ ]:
# Deep Learning Libraries
import tensorflow as tf
import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras import initializers
from keras.wrappers.scikit_learn import KerasClassifier

**Dataset 1 **  
_Data pre-processing and cleaning_

In [ ]:
df1 = pd.read_csv("sgemm_product.csv")
df1.describe()
df1['y']=(df1['Run1 (ms)']+df1['Run2 (ms)']+df1['Run3 (ms)']+df1['Run4 (ms)'])/4

#print(df['y'])
# remove extra fields in data set
df1.drop(['Run1 (ms)', 'Run2 (ms)','Run3 (ms)', 'Run4 (ms)'], axis=1, inplace=True)

target_col = ['y']
matrix_col = ['MWG','NWG']
dimn_col = ['KWG']
wkgsize_col = ['MDIMC','NDIMC']
memshape_col = ['MDIMA','NDIMB']
kloop_col = ['KWI']
vecwidth_col = ['VWM','VWN']
enstride_col = ['STRM','STRN']
cach_col = ['SA','SB']

dep_var = target_col
indep_var = matrix_col + dimn_col+wkgsize_col+memshape_col+kloop_col+vecwidth_col+enstride_col+cach_col
df1.describe()


In [ ]:
# remove outliers

df1 = df1[~(np.abs(df1['y']-df1['y'].mean()) > (3*df1['y'].std()))]#count= 5251 = 2.17% of data
df1.describe()

In [ ]:
df1 = df1.sample(n=100000, random_state=1111)
df1.describe()

**Feature Scaling**

In [ ]:
#Splitting dataset into X and y 
X_Emp = df1.iloc[:,1:].values
y_Emp = df1.iloc[:,0].values

In [ ]:
from sklearn.preprocessing import StandardScaler
full_xscaler = StandardScaler().fit(X_Emp)
x_scaled_Emp = full_xscaler.transform(X_Emp)

In [ ]:
x_scaled_Emp.shape

In [ ]:
pca = PCA(n_components=.80,random_state=123456)
x_scaled_pca_Emp = pca.fit_transform(x_scaled_Emp)
x_scaled_pca_Emp.shape

# K Means Clustering with all feature

In [ ]:
wcss = []

for i in range(1,16):
    Emp_kmeans = KMeans(n_clusters = i,n_jobs = -1,n_init = 10,max_iter = 300,random_state = 0)
    Emp_kmeans.fit(x_scaled_Emp)
    wcss.append(Emp_kmeans.inertia_)
    
plt.plot(range(1,16),wcss)
plt.xlabel('number of clusters')
plt.ylabel('WCSS')
plt.title('Using Elbow method to get the error')
plt.show()

# Best K means with all features Scaled

In [ ]:
Emp_kmeans = KMeans(n_clusters = 4,n_jobs = -1,n_init = 10,max_iter = 300)
y_Emp_kmeans = Emp_kmeans.fit_predict(x_scaled_Emp)

In [ ]:
y_Emp_kmeans

In [ ]:
kmeans_cent = Emp_kmeans.cluster_centers_
kmeans_cent
kmeans_cent.shape

In [ ]:
X_Emp

In [ ]:
n_clusters = 4
kmeans_cent = Emp_kmeans.cluster_centers_
colors = ['#FF0000','#0000FF','#00ff00','#ffff00','#008000','#00fff9' ,'#800080','#FFA500','#16e739','#ff00f4','#FFFF00', '#000000' ,'#FFD700','#808000','#b266b2','#E3CF57']
labels = Emp_kmeans.labels_
col_map=dict(zip(set(labels),colors))
label_color = [col_map[l] for l in labels]
col1 =0
col2 =1


plt.figure(figsize = (10,8))
for i in range(n_clusters):
    plt.scatter(x_scaled_Emp[labels==i, col1],x_scaled_Emp[labels==i, col2],marker='.', s=30, lw=0, alpha=0.9, color = colors[i],edgecolor='k')
    plt.scatter(kmeans_cent[i, col1], kmeans_cent[i, col2], c=colors[i], marker='o',alpha=1, s=200, edgecolor='k', linewidths=2,label = f'Cluster {i}')
    plt.xlabel(f'axis {col1}')
    plt.ylabel(f'axis {col2}')
    plt.legend(loc='best')
plt.title(f'Scaled Dataset with {n_clusters} clusters along with All Features' )

# K Means Clustering on feature selection

In [ ]:
#best Decision tree
deci_tree = tree.DecisionTreeClassifier(max_depth=7, criterion = 'entropy')
deci_tree.fit(x_scaled_Emp,y_Emp)

In [ ]:
x_scaled_Emp.shape

In [ ]:
df1.shape

In [ ]:
exclude_cols = ['y']
deci_tree_impor = pd.DataFrame(deci_tree.feature_importances_, index=[df1[df1.columns[~df1.columns.isin(exclude_cols)]].columns], columns=['importance']).sort_values(by='importance', ascending=False)
deci_tree_impor = deci_tree_impor.reset_index().rename(columns={'level_0': 'features'}, level=0)
imp_feats = deci_tree_impor[deci_tree_impor.importance > 0].features.tolist()
deci_tree_impor[deci_tree_impor.importance > 0]

In [ ]:
# find rows where feature importance is greater than 0
x_scaled_Emp_deciimpfeatures = x_scaled_Emp[:,deci_tree.feature_importances_ > 0]

** Finding optimum no of clusters to be used**

In [ ]:
wcss_imp_features = []

for i in range(1,16):
    Emp_impfeatures_kmeans = KMeans(n_clusters = i,n_jobs = -1,n_init = 10,max_iter = 300,random_state = 0)
    Emp_impfeatures_kmeans.fit(x_scaled_Emp_deciimpfeatures)
    wcss_imp_features.append(Emp_impfeatures_kmeans.inertia_)
    
plt.plot(range(1,16),wcss_imp_features)
plt.xlabel('number of clusters')
plt.ylabel('WCSS')
plt.title('Using Elbow method to get the error for the best features using Decision trees K Means')
plt.show()

# Best K Means feature selection via Decissin Tree

In [ ]:
Emp_impfeatures_kmeans = KMeans(n_clusters = 7,n_jobs = -1,n_init = 10,max_iter = 300)
Emp_impfeatures_kmeans.fit_predict(x_scaled_Emp_deciimpfeatures)

In [ ]:
kmeans_impfeatures_center = Emp_impfeatures_kmeans.cluster_centers_
colors = ['#FF0000','#0000FF','#00ff00','#ffff00','#008000','#00fff9' ,'#800080','#FFA500','#16e739','#ff00f4','#FFFF00', '#000000' ,'#FFD700','#808000','#b266b2']
labels = Emp_impfeatures_kmeans.labels_
color_matrix=dict(zip(set(labels),colors))
label_color = [color_matrix[l] for l in labels]
col1 =0
col2 =1


plt.figure(figsize = (10,8))
for i in range(0,6):
    plt.scatter(x_scaled_Emp_deciimpfeatures[labels==i, col1],x_scaled_Emp_deciimpfeatures[labels==i, col2],marker='.', s=30, lw=0, alpha=0.9, color = colors[i],edgecolor='k')
    plt.scatter(kmeans_impfeatures_center[i, col1], kmeans_impfeatures_center[i, col2], c=colors[i], marker='o',alpha=1, s=200, edgecolor='k', linewidths=2,label = f'Cluster {i}')
    plt.xlabel(f'axis {col1}')
    plt.ylabel(f'axis {col2}')
    plt.legend(loc='best')
plt.title(f'Dataset with 6 for best features extracted using decision tree' )

# Dimensionality Reduction -PCA- K-Means

In [ ]:
wcss_pca = []

for i in range(1,16):
    Emp_pca_kmeans = KMeans(n_clusters = i,n_jobs = -1,n_init = 10,max_iter = 300,random_state = 0)
    Emp_pca_kmeans.fit(x_scaled_pca_Emp)
    wcss_pca.append(Emp_pca_kmeans.inertia_)
    
plt.plot(range(1,16),wcss_pca)
plt.xlabel('number of clusters')
plt.ylabel('WCSS')
plt.title('Using Elbow method to get the error for the PCA K Means')
plt.show()

# Bset K-Means With All features Scaled and PCA applied

In [ ]:
Emp_pca_kmeans = KMeans(n_clusters = 6,n_jobs = -1,n_init = 10,max_iter = 300)
Emp_pca_kmeans.fit_predict(x_scaled_pca_Emp)

In [ ]:
kmeans_pca_center = Emp_pca_kmeans.cluster_centers_
colors = ['#FF0000','#0000FF','#00ff00','#ffff00','#008000','#00fff9' ,'#800080','#FFA500','#16e739','#ff00f4','#FFFF00', '#000000' ,'#FFD700','#808000','#b266b2']
labels = Emp_pca_kmeans.labels_
color_matrix=dict(zip(set(labels),colors))
label_color = [color_matrix[l] for l in labels]
col1 =0
col2 =1


plt.figure(figsize = (10,8))
for i in range(0,6):
    plt.scatter(x_scaled_pca_Emp[labels==i, col1],x_scaled_pca_Emp[labels==i, col2],marker='.', s=30, lw=0, alpha=0.9, color = colors[i],edgecolor='k')
    plt.scatter(kmeans_pca_center[i, col1], kmeans_pca_center[i, col2], c=colors[i], marker='o',alpha=1, s=200, edgecolor='k', linewidths=2,label = f'Cluster {i}')
    plt.xlabel(f'axis {col1}')
    plt.ylabel(f'axis {col2}')
    plt.legend(loc='best')
plt.title(f'Dataset with 7 Scaled All Features and PCA applied' )

# Dimensionality Reductions- ICA -K-means

In [ ]:
# Compute ICA
random_state = 12345
ica = FastICA(n_components=15, random_state=random_state)
x_scaled_ica_Emp = ica.fit_transform(x_scaled_Emp)
ica.mixing_

In [ ]:
wcss_ica = []

for i in range(1,20):
    Emp_ica_kmeans = KMeans(n_clusters = i,n_jobs = -1,n_init = 10,max_iter = 300,random_state = 0)
    Emp_ica_kmeans.fit(x_scaled_ica_Emp)
    wcss_ica.append(Emp_ica_kmeans.inertia_)
    
plt.plot(range(1,20),wcss_ica)
plt.xlabel('number of clusters')
plt.ylabel('WCSS')
plt.title('Using Elbow method to get the error for the ICA K Means')
plt.show()

# Best K MEANS with all features scaled and ICA applied

In [ ]:
Emp_ica_kmeans = KMeans(n_clusters = 7,n_jobs = -1,n_init = 10,max_iter = 300)
Emp_ica_kmeans.fit_predict(x_scaled_ica_Emp)

In [ ]:
kmeans_ica_center = Emp_ica_kmeans.cluster_centers_
colors = ['#FF0000','#0000FF','#00ff00','#ffff00','#008000','#00fff9' ,'#800080','#FFA500','#16e739','#ff00f4','#FFFF00', '#000000' ,'#FFD700','#808000','#b266b2']
labels = Emp_ica_kmeans.labels_
color_matrix=dict(zip(set(labels),colors))
label_color = [color_matrix[l] for l in labels]
col1 =0
col2 =1


plt.figure(figsize = (10,8))
for i in range(0,7):
    plt.scatter(x_scaled_ica_Emp[labels==i, col1],x_scaled_ica_Emp[labels==i, col2],marker='.', s=30, lw=0, alpha=0.9, color = colors[i],edgecolor='k')
    plt.scatter(kmeans_ica_center[i, col1], kmeans_ica_center[i, col2], c=colors[i], marker='o',alpha=1, s=200, edgecolor='k', linewidths=2,label = f'Cluster {i}')
    plt.xlabel(f'axis {col1}')
    plt.ylabel(f'axis {col2}')
    plt.legend(loc='best')
plt.title(f'Dataset with 12 Scaled All Features and ICA applied' )

# Dimensionality Reduction -RCA- KMeans

In [ ]:
# Compute RCA
rca = GaussianRandomProjection(n_components = 20, random_state= random_state)
x_scaled_rca_Emp= rca.fit_transform(x_scaled_Emp)

In [ ]:
wcss_rca = []

for i in range(1,16):
    Emp_rca_kmeans = KMeans(n_clusters = i,n_jobs = -1,n_init = 10,max_iter = 300,random_state = 0)
    Emp_rca_kmeans.fit(x_scaled_rca_Emp)
    wcss_rca.append(Emp_rca_kmeans.inertia_)
    
plt.plot(range(1,16),wcss_rca)
plt.xlabel('number of clusters')
plt.ylabel('WCSS')
plt.title('Using Elbow method to get the error for the RCA K Means')
plt.show()

In [ ]:
Emp_rca_kmeans = KMeans(n_clusters = 4,n_jobs = -1,n_init = 10,max_iter = 300)
Emp_rca_kmeans.fit_predict(x_scaled_rca_Emp)

In [ ]:
kmeans_rca_center = Emp_rca_kmeans.cluster_centers_
colors = ['#FF0000','#0000FF','#00ff00','#ffff00','#008000','#00fff9' ,'#800080','#FFA500','#16e739','#ff00f4','#FFFF00', '#000000' ,'#FFD700','#808000','#b266b2']
labels = Emp_rca_kmeans.labels_
color_matrix=dict(zip(set(labels),colors))
label_color = [color_matrix[l] for l in labels]
col1 =0
col2 =1


plt.figure(figsize = (10,8))
for i in range(0,4):
    plt.scatter(x_scaled_rca_Emp[labels==i, col1],x_scaled_rca_Emp[labels==i, col2],marker='.', s=30, lw=0, alpha=0.9, color = colors[i],edgecolor='k')
    plt.scatter(kmeans_rca_center[i, col1], kmeans_rca_center[i, col2], c=colors[i], marker='o',alpha=1, s=200, edgecolor='k', linewidths=2,label = f'Cluster {i}')
    plt.xlabel(f'axis {col1}')
    plt.ylabel(f'axis {col2}')
    plt.legend(loc='best')
plt.title(f'Dataset with 4 Scaled All Features and RCA applied' )

# Implementing Expectation Minimization

In [ ]:
bic = []
aic = []
convergence_types = ['spherical', 'tied', 'diag', 'full']   
for j in convergence_types:
    for i in range(1,16):
        # Fit a Gaussian mixture with EM
        gmm = GaussianMixture(n_components=i,covariance_type=j, n_init = 10)
        gmm.fit(x_scaled_Emp)
        bic.append(gmm.bic(x_scaled_Emp))
        aic.append(gmm.aic(x_scaled_Emp))

In [ ]:
range_ = range(1, 16)
plt.figure(figsize = (10,8))
for i, convergence_type in enumerate(convergence_types):
    plt.plot(list(range_),bic[i*len(range_):(i+1)*len(range_)],linestyle = '--', label = f'{convergence_type} - AIC')
    plt.plot(list(range_),aic[i*len(range_):(i+1)*len(range_)],label = f'{convergence_type} - BIC')
    plt.legend(loc='best')
    plt.title('AIC & BIC Scores')
    plt.ylabel('AIC & BIC Scores')
    plt.xlabel('number of clusters')

# Best Expectation minimization Full dataset

In [ ]:
Emp_gmm = GaussianMixture(n_components=7 ,covariance_type='full',n_init=10)
Emp_gmm_full = Emp_gmm.fit(x_scaled_Emp).predict(x_scaled_Emp)
means = Emp_gmm.means_

In [ ]:
colors = ['#FF0000','#0000FF','#00ff00','#ffff00','#008000','#00fff9' ,'#800080','#FFA500','#16e739','#ff00f4','#FFFF00', '#000000' ,'#FFD700','#808000','#b266b2']
col_map=dict(zip(set(Emp_gmm_full),colors))
label_color = [col_map[l] for l in Emp_gmm_full]
col1 = 0
col2 = 1

plt.figure(figsize = (10,8))
for i in range(0,7):
    plt.scatter(x_scaled_Emp[Emp_gmm_full==i, col1],x_scaled_Emp[Emp_gmm_full==i, col2],marker='.', s=30, lw=0, alpha=0.7, color = colors[i],edgecolor='k')
    plt.scatter(means[i, col1], means[i, col2], c=colors[i], marker='o',alpha=1, s=200, edgecolor='k', linewidths=2, label = f'Cluster {i}')
    plt.xlabel(f'axis {col1}')
    plt.ylabel(f'axis {col2}')
    plt.legend(loc='best')


plt.title(f'Dataset with 7 Scaled Expec Max and all features' )

# Expectation maximization on feature scaling

In [ ]:
bic_impfeatures = []
aic_impfeatures = []
convergence_types = ['spherical', 'tied', 'diag', 'full']   
for j in convergence_types:
    for i in range(1,9):
        # Fit a Gaussian mixture with EM
        gmm_impfeatures = GaussianMixture(n_components=i,covariance_type=j, n_init = 8)
        gmm_impfeatures.fit(x_scaled_Emp_deciimpfeatures)
        bic_impfeatures.append(gmm.bic(x_scaled_Emp_deciimpfeatures))
        aic_impfeatures.append(gmm.aic(x_scaled_Emp_deciimpfeatures))

In [ ]:
range_ = range(1, 16)
plt.figure(figsize = (10,8))
for i, convergence_type in enumerate(convergence_types):
    plt.plot(list(range_),bic_impfeatures[i*len(range_):(i+1)*len(range_)],linestyle = '--', label = f'{convergence_type} - AIC')
    plt.plot(list(range_),aic_impfeatures[i*len(range_):(i+1)*len(range_)],label = f'{convergence_type} - BIC')
    plt.legend(loc='best')
    plt.title('AIC & BIC Scores')
    plt.ylabel('AIC & BIC Scores')
    plt.xlabel('number of clusters')

# Best Expectation Maximization on important feature selected dataset

In [ ]:
Emp_gmm_imp = GaussianMixture(n_components=6 ,covariance_type='full',n_init=10)
Emp_gmm_impfeatures = Emp_gmm_imp.fit(x_scaled_Emp_deciimpfeatures).predict(x_scaled_Emp_deciimpfeatures)
means_impfeatures = Emp_gmm_imp.means_

In [ ]:
colors = ['#FF0000','#0000FF','#00ff00','#ffff00','#008000','#00fff9' ,'#800080','#FFA500','#16e739','#ff00f4','#FFFF00', '#000000' ,'#FFD700','#808000','#b266b2']
col_map=dict(zip(set(Emp_gmm_impfeatures),colors))
label_color = [col_map[l] for l in Emp_gmm_impfeatures]
col1 = 0
col2 = 1

plt.figure(figsize = (10,8))
for i in range(0,6):
    plt.scatter(x_scaled_Emp_deciimpfeatures[Emp_gmm_impfeatures==i, col1],x_scaled_Emp_deciimpfeatures[Emp_gmm_impfeatures==i, col2],marker='.', s=30, lw=0, alpha=0.7, color = colors[i],edgecolor='k')
    plt.scatter(means_impfeatures[i, col1], means_impfeatures[i, col2], c=colors[i], marker='o',alpha=1, s=200, edgecolor='k', linewidths=2, label = f'Cluster {i}')
    plt.xlabel(f'axis {col1}')
    plt.ylabel(f'axis {col2}')
    plt.legend(loc='best')


plt.title(f'Dataset with 6 Scaled Expec Max and all features' )

# Dimensionality reduction -PCA -Expectation Maximization

In [ ]:
bic_pca = []
aic_pca = []
convergence_types = ['spherical', 'tied', 'diag', 'full']   
for j in convergence_types:
    for i in range(1,16):
        # Fit a Gaussian mixture with EM
        gmm_pca = GaussianMixture(n_components=i,covariance_type=j, n_init = 10)
        gmm_pca.fit(x_scaled_pca_Emp)
        bic_pca.append(gmm_pca.bic(x_scaled_pca_Emp))
        aic_pca.append(gmm_pca.aic(x_scaled_pca_Emp))

In [ ]:
range_ = range(1, 16)
plt.figure(figsize = (10,8))
for i, convergence_type in enumerate(convergence_types):
    plt.plot(list(range_),bic_pca[i*len(range_):(i+1)*len(range_)],linestyle = '--', label = f'{convergence_type} - AIC')
    plt.plot(list(range_),aic_pca[i*len(range_):(i+1)*len(range_)],label = f'{convergence_type} - BIC')
    plt.legend(loc='best')
    plt.title('AIC & BIC Scores')
    plt.ylabel('AIC & BIC Scores')
    plt.xlabel('number of clusters')

# Best Expectation-Maximization on PCA applied Dataset 

In [ ]:
Emp_gmm_pca = GaussianMixture(n_components=6 ,covariance_type='full',n_init=10)
Emp_gmm_pcafeatures = Emp_gmm_pca.fit(x_scaled_pca_Emp).predict(x_scaled_pca_Emp)
means_pcafeatures = Emp_gmm_pca.means_

In [ ]:
colors = ['#FF0000','#0000FF','#00ff00','#ffff00','#008000','#00fff9' ,'#800080','#FFA500','#16e739','#ff00f4','#FFFF00', '#000000' ,'#FFD700','#808000','#b266b2']
col_map=dict(zip(set(Emp_gmm_pcafeatures),colors))
label_color = [col_map[l] for l in Emp_gmm_pcafeatures]
col1 = 0
col2 = 1

plt.figure(figsize = (10,8))
for i in range(0,6):
    plt.scatter(x_scaled_pca_Emp[Emp_gmm_pcafeatures==i, col1],x_scaled_pca_Emp[Emp_gmm_pcafeatures==i, col2],marker='.', s=30, lw=0, alpha=0.7, color = colors[i],edgecolor='k')
    plt.scatter(means_pcafeatures[i, col1], means_pcafeatures[i, col2], c=colors[i], marker='o',alpha=1, s=200, edgecolor='k', linewidths=2, label = f'Cluster {i}')
    plt.xlabel(f'axis {col1}')
    plt.ylabel(f'axis {col2}')
    plt.legend(loc='best')


plt.title(f'Dataset with 6 Scaled Expec Max and all features' )

# Dimensionality Reductions - ICA - Expectation Maximization

In [ ]:
bic_ica = []
aic_ica = []
convergence_types = ['spherical', 'tied', 'diag', 'full']   
for j in convergence_types:
    for i in range(1,16):
        # Fit a Gaussian mixture with EM
        gmm_ica = GaussianMixture(n_components=i,covariance_type=j, n_init = 10)
        gmm_ica.fit(x_scaled_ica_Emp)
        bic_ica.append(gmm_ica.bic(x_scaled_ica_Emp))
        aic_ica.append(gmm_ica.aic(x_scaled_ica_Emp))

In [ ]:
range_ = range(1, 16)
plt.figure(figsize = (10,8))
for i, convergence_type in enumerate(convergence_types):
    plt.plot(list(range_),bic_ica[i*len(range_):(i+1)*len(range_)],linestyle = '--', label = f'{convergence_type} - AIC')
    plt.plot(list(range_),aic_ica[i*len(range_):(i+1)*len(range_)],label = f'{convergence_type} - BIC')
    plt.legend(loc='best')
    plt.title('AIC & BIC Scores')
    plt.ylabel('AIC & BIC Scores')
    plt.xlabel('number of clusters')

# Best Expectation-Maximization on ICA applied Dataset 

In [ ]:
Emp_gmm_ica = GaussianMixture(n_components=6 ,covariance_type='full',n_init=10)
Emp_gmm_icafeatures = Emp_gmm_ica.fit(x_scaled_ica_Emp).predict(x_scaled_ica_Emp)
means_icafeatures = Emp_gmm_ica.means_

In [ ]:
colors = ['#FF0000','#0000FF','#00ff00','#ffff00','#008000','#00fff9' ,'#800080','#FFA500','#16e739','#ff00f4','#FFFF00', '#000000' ,'#FFD700','#808000','#b266b2']
col_map=dict(zip(set(Emp_gmm_icafeatures),colors))
label_color = [col_map[l] for l in Emp_gmm_icafeatures]
col1 = 0
col2 = 1

plt.figure(figsize = (10,8))
for i in range(0,6):
    plt.scatter(x_scaled_ica_Emp[Emp_gmm_icafeatures==i, col1],x_scaled_ica_Emp[Emp_gmm_icafeatures==i, col2],marker='.', s=30, lw=0, alpha=0.7, color = colors[i],edgecolor='k')
    plt.scatter(means_icafeatures[i, col1], means_icafeatures[i, col2], c=colors[i], marker='o',alpha=1, s=200, edgecolor='k', linewidths=2, label = f'Cluster {i}')
    plt.xlabel(f'axis {col1}')
    plt.ylabel(f'axis {col2}')
    plt.legend(loc='best')


plt.title(f'Dataset with 6 Scaled Expec Max and all features' )

# Dimensionality Reductions - RCA - Expectation Maximization

In [ ]:
bic_rca = []
aic_rca = []
convergence_types = ['spherical', 'tied', 'diag', 'full']   
for j in convergence_types:
    for i in range(1,16):
        # Fit a Gaussian mixture with EM
        gmm_rca = GaussianMixture(n_components=i,covariance_type=j, n_init = 10)
        gmm_rca.fit(x_scaled_rca_Emp)
        bic_rca.append(gmm_rca.bic(x_scaled_rca_Emp))
        aic_rca.append(gmm_rca.aic(x_scaled_rca_Emp))

In [ ]:
range_ = range(1, 16)
plt.figure(figsize = (10,8))
for i, convergence_type in enumerate(convergence_types):
    plt.plot(list(range_),bic_rca[i*len(range_):(i+1)*len(range_)],linestyle = '--', label = f'{convergence_type} - AIC')
    plt.plot(list(range_),aic_rca[i*len(range_):(i+1)*len(range_)],label = f'{convergence_type} - BIC')
    plt.legend(loc='best')
    plt.title('AIC & BIC Scores')
    plt.ylabel('AIC & BIC Scores')
    plt.xlabel('number of clusters')

# Best Expectation-Maximization on RCA applied Dataset 

In [ ]:
Emp_gmm_rca = GaussianMixture(n_components=6 ,covariance_type='full',n_init=10)
Emp_gmm_rcafeatures = Emp_gmm_rca.fit(x_scaled_rca_Emp).predict(x_scaled_rca_Emp)
means_rcafeatures = Emp_gmm_rca.means_

In [ ]:
colors = ['#FF0000','#0000FF','#00ff00','#ffff00','#008000','#00fff9' ,'#800080','#FFA500','#16e739','#ff00f4','#FFFF00', '#000000' ,'#FFD700','#808000','#b266b2']
col_map=dict(zip(set(Emp_gmm_rcafeatures),colors))
label_color = [col_map[l] for l in Emp_gmm_rcafeatures]
col1 = 0
col2 = 1

plt.figure(figsize = (10,8))
for i in range(0,6):
    plt.scatter(x_scaled_rca_Emp[Emp_gmm_rcafeatures==i, col1],x_scaled_rca_Emp[Emp_gmm_rcafeatures==i, col2],marker='.', s=30, lw=0, alpha=0.7, color = colors[i],edgecolor='k')
    plt.scatter(means_rcafeatures[i, col1], means_rcafeatures[i, col2], c=colors[i], marker='o',alpha=1, s=200, edgecolor='k', linewidths=2, label = f'Cluster {i}')
    plt.xlabel(f'axis {col1}')
    plt.ylabel(f'axis {col2}')
    plt.legend(loc='best')


plt.title(f'Dataset with 6 Scaled Expec Max and all features' )

# Comparison of Clusters for Dataset Using Homogenity Scores

In [ ]:
y = target_col

homo_scores = []
names = []
homo_scores.append(homogeneity_score(y,Emp_kmeans.labels_))
names.append('kmeans AllFeatures')

homo_scores.append(homogeneity_score(y,Emp_impfeatures_kmeans.labels_))
names.append('kmeans ImpFeat')

homo_scores.append(homogeneity_score(y,Emp_pca_kmeans.labels_))
names.append('kmeans PCA')

homo_scores.append(homogeneity_score(y,Emp_ica_kmeans.labels_))
names.append('kmeans ICA')

homo_scores.append(homogeneity_score(y,Emp_rca_kmeans.labels_))
names.append('kmeans RCA')

homo_scores.append(homogeneity_score(y,Emp_gmm_full))
names.append('ExpecMax AllFeatures')

homo_scores.append(homogeneity_score(y,Emp_gmm_impfeatures))
names.append('ExpecMax ImpFeat')

homo_scores.append(homogeneity_score(y,Emp_gmm_pcafeatures))
names.append('ExpecMax PCA')

homo_scores.append(homogeneity_score(y,Emp_gmm_icafeatures))
names.append('ExpecMax ICA')

homo_scores.append(homogeneity_score(y,Emp_gmm_rcafeatures))
names.append('ExpecMax RCA')

In [ ]:
plt.figure(figsize = (10,10))
y_ = np.arange(len(names))
plt.bar(y_,homo_scores, align='edge')
plt.xticks(y_, names,rotation = 75)
plt.ylabel('Homogenity Scores')
plt.title('Comparison of Various Clustering Type on Datasets')
plt.show()

# Comparing the accuracy rate for all the models

In [ ]:
datasets = {'scaled_dataset': x_scaled_Emp, 'Imp_feat_dataset':x_scaled_Emp_deciimpfeatures, 'PCA_dataset':x_scaled_pca_Emp, 
            'ICA_dataset':x_scaled_ica_Emp,  'RCA_dataset':x_scaled_rca_Emp, 'labels_dataset':Emp_kmeans.labels_}
y = target_col
model_results = []

cv = 8
nodes_hidden_layer1 = 120
nn_cv = []
kfold = KFold(n_splits=cv,shuffle=True)

plt.figure(figsize = (10,8))
for key, dataset in datasets.items():
    if key == 'labels_dataset':
        x_train, x_test, y_train, y_test = train_test_split(dataset, y, test_size = .20, random_state = 0)
        
        try:
            y_test_row, y_test_classes = y_test.shape
        except ValueError:
            y_test_row = y_test.shape[0]
            y_test_classes = 1
            
        x_train = x_train[:,np.newaxis]
        y_train = y_train[:,np.newaxis]
        for train, test in kfold.split(x_train,y_train):
            # earling stopping defined
            es = EarlyStopping(monitor='val_acc', mode='max')
            # Initializing the ANN
            classifier = Sequential()
            #Adding the input layer and the first hidden layer
            classifier.add(Dense(nodes_hidden_layer1, input_shape=(x_train.shape[1],),activation='softmax', kernel_initializer=initializers.random_normal(mean = 0.0, stddev=0.02)))
            for i in range(1, 3):
                nodes_hidden_layer1 = np.ceil(nodes_hidden_layer1/2).astype('int32')
                classifier.add(Dense(nodes_hidden_layer1, activation='relu', kernel_initializer=initializers.random_normal(mean = 0.0, stddev=0.02)))

                # Adding the output layer
            classifier.add(Dense(y_test_classes, activation='sigmoid'))

                # Compiling the ANN
            classifier.compile(loss='binary_crossentropy',optimizer = 'adam', metrics = ['accuracy'])

                # Fitting the ANN to the training set
            history = classifier.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=100, batch_size = 400, verbose=0, callbacks=[es])

                # evaluate the model
            train_loss, train_acc = classifier.evaluate(x_train, y_train, verbose=0)
            test_loss, test_acc = classifier.evaluate(x_test, y_test, verbose=0)
            nn_cv.append(test_acc)
        model_results.append(np.array(nn_cv))
            
    else:
        x_train, x_test, y_train, y_test = train_test_split(dataset, y, test_size = .20, shuffle = True)
        try:
            y_test_row, y_test_classes = y_test.shape
        except ValueError:
            y_test_row = y_test.shape[0]
            y_test_classes = 1
        for train, test in kfold.split(x_train,y_train):
            # earling stopping defined
            es = EarlyStopping(monitor='val_acc', mode='max')
            # Initializing the ANN
            classifier = Sequential()
            #Adding the input layer and the first hidden layer
            classifier.add(Dense(nodes_hidden_layer1, input_shape=(x_train.shape[1],),activation='softmax', kernel_initializer=initializers.random_normal(mean = 0.0, stddev=0.02)))
            for i in range(1, 3):
                nodes_hidden_layer1 = np.ceil(nodes_hidden_layer1/2).astype('int32')
                classifier.add(Dense(nodes_hidden_layer1, activation='relu', kernel_initializer=initializers.random_normal(mean = 0.0, stddev=0.02)))

                # Adding the output layer
            classifier.add(Dense(y_test_classes, activation='sigmoid'))

                # Compiling the ANN
            classifier.compile(loss='binary_crossentropy',optimizer = 'adam', metrics = ['accuracy'])

                # Fitting the ANN to the training set
            history = classifier.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=100, batch_size = 400, verbose=0, callbacks=[es])

                # evaluate the model
            train_loss, train_acc = classifier.evaluate(x_train, y_train, verbose=0)
            test_loss, test_acc = classifier.evaluate(x_test, y_test, verbose=0)
            nn_cv.append(test_acc)
        model_results.append(np.array(nn_cv))

In [ ]:
models = ['scaled_dataset', 'Imp_feat_dataset','PCA_dataset', 'ICA_dataset','RCA_dataset','labels_dataset']
fig = plt.figure(figsize=(15,12))
fig.suptitle('COMPARISON OF VARIOUS MODELS ON EMPLOYEE ATTRITION DATASET')
ax = fig.add_subplot(111)
plt.boxplot(model_results)
ax.set_xticklabels(models)
ax.set_ylabel('Accuracy')